In [77]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup



In [40]:
!apt-get update > /dev/null 2>&1
!pip install selenium > /dev/null 2>&1
!apt install chromium-chromedriver > /dev/null 2>&1

������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.


In [78]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [79]:
import warnings
warnings.filterwarnings('ignore')

In [80]:
driver = webdriver.Chrome()

In [81]:
url = 'https://nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=35.1493479&myY=126.9218479&searchKeyword=&pageIndex=1'
driver.get(url)

In [45]:
# 입력창에 광주를 입력하고 검색 버튼을 누르는 동ㅇ작
driver.find_element_by_id('searchKeyword').send_keys('광주')
driver.find_element_by_id('fsSearchBtn').click()

이동 후에는 BeautifulSoup 

In [82]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li')

len(lis)

10

In [83]:
li = lis[0]

name = li.select_one('.title').get_text().strip()
name

'동부소방서_광주-대인-119 안전센터'

In [88]:
addr = li.find('address').string.strip()
addr

'광주광역시 동구 제봉로 210(대인동)'

In [70]:
tel = li.select_one('.tel').get_text().strip()
tel

'전화공사중'

In [ ]:
lines = []
for li in lis:
    name = li.select_one('.title').get_text().strip()
    addr = li.find('address').string.strip()
    tel = li.select_one('.tel').get_text().strip()
    lines.append([name, addr, tel])

### 두번째 페이지로 이동

//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a

In [ ]:
driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()

### 다음 페이지로 이동

In [ ]:

driver.find_element(By.CSS_SELECTOR, '.next_page').click()

### 전체

In [95]:
# 검색결과 건수 획득
strongs = soup.select('.stations-local-after > p > strong')
strongs[1].get_text()

IndexError: list index out of range

In [ ]:
count = int(strongs[1].get_text().replace('건','').strip())
count

AttributeError: 'NoneType' object has no attribute 'string'

In [64]:
import math
count = math.ceil(count / 10)
count

NameError: name 'count' is not defined

### 최종 코드

In [ ]:
driver.get(url)
time.sleep(1)
driver.find_element(By.ID, 'searchKeyword').send_keys('광주')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [ ]:
strongs = soup.select('.stations-local-after > p > strong')
count = int(strongs[1].get_text().replace('건','').strip())
count = math.ceil(count / 10)

In [ ]:
lines = []
for i in range(count):
    if i >= 1 and i%2 == 0:
        driver.find_element(By.CSS_SELECTOR, '.next_page').click()
        time.sleep(2)
    if i >= 1 and i%2 == 1:
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        time.sleep(2)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list > li')
    for li in lis:
        name = li.select_one('.title').get_text().strip()
        addr = li.find('address').string.strip()
        tel = li.select_one('.tel').get_text().strip()
        lines.append([name, addr, tel]) 


In [ ]:

len(lines)

In [ ]:
df = pd.DataFrame(lines, columns=['관서명','주소', '전화번호'])
df.tail()